# Final Assignment Submission for FMUP
## Kishlaya Jaiswal
### Chennai Mathematical Institute - MCS201909
---

In [17]:
# Necessary Imports

from statistics import NormalDist
from math import exp, log, sqrt
import QuantLib as ql


# Necessary Functions (used later on)

def dist_mu_sigma(S_0, r, vol, T):
    # returns the mu and sigma as required by 
    # the formula for finding the call/put premium
    # in Black-Scholes Model
    
    mu = log(S_0) + T*(r - (vol**2)/2)
    sigma = (T*vol**2)**.5
    return (mu, sigma)


## Problem 1(a)

We will be using the following two securities for this question:
 - HINDUNILVR
 - NESTLEIND
 
The expected returns and annualized volatility computations are given at the end.

We use the Black-Scholes to model the stock-price $S_T$ movements of these equities, that is
$$\ln S_T \xrightarrow{distribution} N\left[\mu, \sigma_0^2\right]$$

where $\mu = \ln S_0 + \left(r + \frac12\sigma^2\right)T, \sigma_0^2 = \sigma^2 T $ and 
 - $S_0$ is initial price of stock
 - $T$ is time period
 - $r$ is expected returns
 - $\sigma$ is volatility

In [18]:
# Closing price of HINDUNILVR on May 17, 2021
S_0_1 = 2376.25

# Expected Return and Annualized Volatility from historical data
r_1 = 18.59/100
vol_1 = 20.61/100

# Time period of one day
T = 1/365

# Find mu and sigma for normal distribution of stock price S_T
mu_1, sigma_1 = dist_mu_sigma(S_0_1, r_1, vol_1, T)

# Generate the normal distribution with the above mu and sigma
nd_1 = NormalDist(mu_1, sigma_1)

# Calculate K such that probability that value of S_T > K is 0.95
# where Maximum Loss = Initial Price - K
K_1 = exp(nd_1.inv_cdf(1 - 0.95))
Loss_1 = S_0_1 - K_1

print("Maximum Loss in a day: {0:.3f}".format(Loss_1))


Maximum Loss in a day: 40.740


## Problem 1(b)

We assume that the two stocks we selected above are independent of each other. Hence the total loss in portfolio is the sum of losses of each stock, provided that we bought equal quantities, say 1 unit of each stock.

In [19]:
# Closing price of NESTLEIND on May 17, 2021
S_0_2 = 17037.95

# Expected Return and Annualized Volatility from historical data
r_2 = 7.25/100
vol_2 = 21.81/100

# Find mu and sigma for normal distribution of stock price S_T
mu_2, sigma_2 = dist_mu_sigma(S_0_2, r_2, vol_2, T)

# Generate the normal distribution with the above mu and sigma
nd_2 = NormalDist(mu_2, sigma_2)

# Calculate K such that probability that value of S_T > K is 0.95
# where Maximum Loss = Initial Price - K
K_2 = exp(nd_2.inv_cdf(1 - 0.95))
Loss_2 = S_0_2 - K_2

print("Maximum Loss in a day: {0:.3f}".format(Loss_1 + Loss_2))

Maximum Loss in a day: 355.452


## Problem 1(c)

Suppose we buy $a$ units of Stock1 and $b$ units of Stock2 such that $$aS_1 + bS_2 = X$$ where $S_1, S_2$ are the spot prices (initial values) of Stock1 and Stock2 respectively.

Assuming Stock1 and Stock2 are independent, we get that $$V(\text{Portfolio}) = a^2 V_1 + b^2 V_2$$ where $V_1, V_2$ are variances of Stock1 and Stock2 respectively.

So now our goal is to:
\begin{align*}
\min \quad a^2 V_1 + b^2 V_2 \\
\text{such that } aS_1 + bS_2 = X \\
\text{and } 0 \leq a, b \leq X
\end{align*}

Now we note that since $S_2^2 > 0$ minimizing $a^2 V_1 + b^2 V_2$ is same as minimizing $S_2^2(a^2 V_1 + b^2 V_2)$
\begin{align*}
S_2^2(a^2 V_1 + b^2 V_2) &= (S_2a)^2 V_1 + (S_2b)^2 V_2 \\
&= (S_2a)^2 V_1 + (X - S_1a)^2 V_2 \\
&= (S_2^2 V_1 + S_1^2 V_2)a^2 - (2S_1XV_2)a + X^2 V_2 \\
\end{align*}

which is a quadratic equation (in variable $a$) whose minimum value is attained at $$a = \frac{S_1 V_2}{S_2^2 V_1 + S_1^2 V_2} X \implies b = \frac{S_2 V_1}{S_2^2 V_1 + S_1^2 V_2} X$$

And these values give the minimum variance of portfolio as $$V_{\min} = \frac{V_1V_2}{S_2^2 V_1 + S_1^2 V_2}X^2$$


## Problem 2

Recall that, in Black-Scholes model, value of the call option is given as:

$$C = S_0 \phi(d_1) - Ke^{-rT}\phi(d_2)$$

where $$d_1 = \frac{\ln (\frac{S_0}{K}) + (r + \frac12\sigma^2)T}{\sigma \sqrt{T}}$$
$$d_2 = d_1 - \sigma \sqrt{T}$$

In [20]:
# Initial/Spot price
S_0 = 42

# Exercise price
K = 40

# risk-free interest rate
r = 10/100

# volatility
sigma = 20/100

# Time period
t = 6/12

# d1 and d2 are calculated as in above formula
d1 = (log(S_0/K) + (r + (0.5*sigma*sigma))*t)/(sigma * sqrt(t))
d2 = d1 - sigma*sqrt(t)

nd = NormalDist(0,1)
call = S_0 * nd.cdf(d1) - K * exp(-r*t)*nd.cdf(d2)

print("Price of Call Option: {0:.3f}".format(call))

Price of Call Option: 4.759


## Problem 3

We first interpolate to obtain values of spot-rates at the required time periods.

Having obtained the spot rates, we simply use the following formulas to obtain current price $P$, duration $D$ and convexity $C$ of the given bond.

$$P = \sum_{i=1}^n \frac{C_i}{(1+y_i)^i} + \frac{F}{(1+y_n)^n}$$

$$D = \frac{1}{P} \left(\sum_{i=1}^n \frac{iC_i}{(1+y_i)^{i+1}} + \frac{nF}{(1+y_n)^{n+1}}\right)$$

$$C = \frac{1}{P} \left(\sum_{i=1}^n \frac{i(i+1)C_i}{(1+y_i)^{i+2}} + \frac{n(n+1)F}{(1+y_n)^{n+2}}\right)$$

In [21]:
# Create the interpolation function to obtain values of spot rates at periods not specified in table
# from the following given values 
# Here X is the time period
# And Y is the spot rate at that period
X = [0.5, 1, 2, 3, 5, 10, 20]
Y = [2.9, 4.4, 4.8, 5.0, 5.3, 5.4, 5.5]

i = ql.LinearInterpolation(X, Y)

# Coupon Rate
rate = 0.08/2

# Face Value
fv = 100

# Time periods of coupon payment
dates = [x/2 for x in range(1,41)]

# Last coupon payment rate
yn = i(dates[-1])/100

# Number of payments
n = len(dates)

# Semin-annual payments
m = 2

value = 0
for index, d in enumerate(dates):
    y = i(d)/100
    value += rate*fv / (1 + y/m)**(index + 1)
value += fv / (1 + yn/m)**n

print("Current Value of the Bond: {0:.3f}".format(value))


duration = 0
for index, d in enumerate(dates):
    y = i(d)/100
    duration += (index + 1)*rate*fv / (1 + y/m)**(index + 2)
duration += fv*n / (1 + yn/m)**(n+1)
duration = duration / (2 * value)

print("Duration of the Bond: {0:.3f} years".format(duration))


convexity = 0
for index, d in enumerate(dates):
    y = i(d)/100
    convexity += (index + 1)*(index + 2)*rate*fv / (1 + y/m)**(index + 3)
convexity += fv*n*(n+1) / (1 + yn/m)**(n+2)
convexity = convexity / (4 * value)

print("Convexity of the Bond: {0:.3f}".format(convexity))

Current Value of the Bond: 131.066
Duration of the Bond: 11.132 years
Convexity of the Bond: 175.506


## Problem 4

So we have, $C = 6, P = 3, K = 95, S_0 = 94, r = 0.01, T = 4/12$ 

Put-Call Parity says that: to avoid riskless arbitrage, $C + Ke^{-rT} = P + S_0$

We check that put-call parity is not satisfied.

In [22]:
C, P, K, S_0, r, T = 6,3, 95, 94, 0.01, 4/12
X = C + K*exp(-r * T)
Y = P + S_0
print(X > Y)

True


And so we have in fact, $C + Ke^{-rT} > S_0 + P$

Using this, we construct ourportfolio as follows:
 - Short 1 Call Option
 - Borrow $Ke^{-rT}$
 - Buy 1 unit of stock
 - Buy 1 Put Option
 
At time $t=0$, value of our portfolio is: $+C + Ke^{-rT} - S_0 - P > 0$

Now suppose at time $t=T$, $S_T < K$ then value of our portfolio is: $0 - E + S_T + (E - S_T) = 0$

And if at time $t=T$, $S_T \geq K$ then value of our portfolio is: $(E - S_T) - E + S_T + 0 = 0$

In any case, we have obtained a riskless arbitrage profit.

## Problem 5

Current price of stock is $50$.
So in first step, it could be either $60$ or $40$
So in second step, it could be either $72$ or $48$ or $32$

The exercise price is $55$ so call option can be exercised only if the value of stock is at $72$ in which the payoff is $17$ and in all other cases payoff is $0$

Recall that in the 2-step binomial, current price of call option is given by:
$$f_0 = e^{-rT}[qf_u + (1-q)f_d]$$

where 
$$q = \frac{e^{rT} - d}{u-d}$$
$$f_u = e^{-rT} [qf_{uu} + (1-q)f_{ud}]$$
$$f_d = e^{-rT} [qf_{du} + (1-q)f_{dd}]$$

and $f_{uu} = 17, f_{ud} = f_{du} = f_{dd} = 0$

In [23]:
r = 5/100
T = 1
K = 55
S_0 = 50
u = 1.2
d = 0.8

f_uu = max(0, S_0*u*u - K)
f_ud = f_du = max(0, S_0*u*d - K)
f_dd = max(0, S_0*d*d - K)

q = (exp(r*T) - d)/(u-d)
f_u = exp(-r*T)*(q*f_uu + (1-q)*f_ud)
f_d = exp(-r*T)*(q*f_ud + (1-q)*f_dd)

f_0 = exp(-r*T)*(q*f_u + (1-q)*f_d)

print("Current Price of Call Option: {0:.3f}".format(f_0))

Current Price of Call Option: 6.070


## Problem 6

To be able to exercise the call option, we must have that $S_T > K \implies \ln S_T > \ln K$

Since $\ln S_T \xrightarrow{distribution} N\left[\mu, \sigma_0^2\right]$ as explained in question 1, so let cdf (cumulative distributive function) of $\ln S_T$ be $\phi$, then we get that

$$P(\text{Exercise}) = P(S_T > K) = P(\ln S_T > \ln K) = 1 - P(\ln S_T \leq \ln K) = 1 - \phi(\ln K)$$

In [24]:
S_0 = 38
K = 40
r = 10/100
vol = 25/100
T = 6/12

mu = log(S_0) + T*(r - (vol**2)/2)
sigma = (T*vol**2)**.5

nd = NormalDist(mu, sigma)
exer_prob = 1 - nd.cdf(log(K))
print("Exercise Probability of Call Option: {0:.3f}".format(exer_prob))

Exercise Probability of Call Option: 0.462


## Problem 7

Recall that, in Black-Scholes model, value of the put option is given as:

$$P = Ke^{-rT}\phi(-d_2) - S_0 \phi(-d_1)$$

where $$d_1 = \frac{\ln (\frac{S_0}{K}) + (r + \frac12\sigma^2)T}{\sigma \sqrt{T}}$$
$$d_2 = d_1 - \sigma \sqrt{T}$$

In [25]:
S_0 = 69
K = 70
r = 5/100
sigma = 25/100
T = 6/12
d1 = (log(S_0/K) + (r + (0.5*sigma*sigma))*T)/(sigma * sqrt(T))
d2 = d1 - sigma*sqrt(T)

nd = NormalDist(0,1)
put = K * exp(-r*t)*nd.cdf(-d2) - S_0 * nd.cdf(-d1)
print("Price of Put Option: {0:.3f}".format(put))

Price of Put Option: 4.479


### Return and Volatility Computation for Problem 1

Data obtained from Yahoo Finance

In [26]:
def get_volatility(csv):
    data = csv.split('\n')[1:]
    data = map(lambda x: x.split(','), data)
    closing_prices = list(map(lambda x: float(x[-2]), data))
    
    n = len(closing_prices)
    
    log_returns = []
    for i in range(1,n):
        log_returns.append(log(closing_prices[i]/closing_prices[i-1]))
    
    mu = sum(log_returns)/(n-1)
    
    tmp = map(lambda x: (x-mu)**2, log_returns)
    
    vol = sqrt(sum(tmp)/(n-1)) * sqrt(252)
    return vol

def get_return(csv):
    data = csv.split('\n')[1:]
    data = map(lambda x: x.split(','), data)
    closing_prices = list(map(lambda x: float(x[-2]), data))
    init = closing_prices[0]
    final = closing_prices[-1]
    mu = log(final/init)
    return mu

hind_csv = '''Date,Open,High,Low,Close,Adj Close,Volume
2020-05-18,2041.500000,2047.849976,1988.599976,2005.250000,1970.422241,7675561
2020-05-19,2018.000000,2024.800049,1968.099976,1975.550049,1941.238159,5963193
2020-05-20,1975.550049,2010.000000,1975.000000,1990.849976,1956.272461,3926496
2020-05-21,1999.000000,2003.400024,1955.199951,1970.900024,1936.668823,4261625
2020-05-22,1975.000000,1995.050049,1958.000000,1987.900024,1953.373657,5339946
2020-05-26,2004.949951,2020.000000,1975.000000,1977.800049,1943.449219,4628932
2020-05-27,1990.000000,2005.099976,1961.000000,1998.099976,1963.396484,4261645
2020-05-28,2000.000000,2015.000000,1990.000000,2009.949951,1975.040649,5500796
2020-05-29,2009.000000,2068.000000,2000.000000,2057.350098,2021.617554,8778345
2020-06-01,2075.000000,2143.000000,2071.199951,2108.550049,2071.928223,6592547
2020-06-02,2122.000000,2125.000000,2090.000000,2107.399902,2070.798340,3268728
2020-06-03,2115.000000,2149.000000,2085.000000,2120.500000,2083.670654,4895117
2020-06-04,2115.000000,2132.399902,2105.250000,2120.600098,2083.768799,3264730
2020-06-05,2124.000000,2128.000000,2085.000000,2087.600098,2051.342041,2981853
2020-06-08,2107.000000,2124.500000,2078.100098,2111.699951,2075.023438,4298456
2020-06-09,2100.000000,2144.800049,2095.050049,2106.500000,2069.913574,3127904
2020-06-10,2115.899902,2130.000000,2096.000000,2119.600098,2082.786377,2421659
2020-06-11,2115.000000,2138.199951,2095.649902,2106.899902,2070.306641,2566927
2020-06-12,2065.250000,2119.000000,2065.000000,2107.250000,2070.650879,2338438
2020-06-15,2114.850098,2131.699951,2056.800049,2084.300049,2048.099365,2830788
2020-06-16,2109.800049,2120.000000,2057.000000,2076.350098,2040.287598,2806772
2020-06-17,2076.000000,2099.000000,2062.300049,2078.250000,2042.154541,2293301
2020-06-18,2070.250000,2084.850098,2054.050049,2065.350098,2029.478394,2326907
2020-06-19,2079.000000,2100.000000,2057.000000,2092.750000,2070.437256,4673528
2020-06-22,2110.000000,2118.899902,2087.000000,2100.800049,2078.401367,2487234
2020-06-23,2115.000000,2172.000000,2101.600098,2155.949951,2132.963379,4308814
2020-06-24,2172.000000,2176.899902,2125.000000,2130.300049,2107.586670,3023015
2020-06-25,2114.300049,2189.899902,2112.000000,2173.149902,2149.979736,8208033
2020-06-26,2179.949951,2179.949951,2138.100098,2154.199951,2131.231934,3176048
2020-06-29,2151.350098,2195.899902,2140.000000,2182.949951,2159.675293,3295039
2020-06-30,2193.800049,2199.949951,2170.000000,2180.000000,2156.756836,2373109
2020-07-01,2183.000000,2191.800049,2160.100098,2170.399902,2147.259033,1532931
2020-07-02,2167.750000,2177.649902,2145.000000,2151.750000,2128.808105,2588838
2020-07-03,2165.500000,2188.000000,2156.350098,2173.699951,2150.523926,1946252
2020-07-06,2180.000000,2180.000000,2153.000000,2161.600098,2138.553223,2375516
2020-07-07,2175.500000,2175.500000,2151.649902,2154.149902,2131.182373,2014667
2020-07-08,2153.600098,2194.000000,2125.000000,2186.050049,2162.742432,3349938
2020-07-09,2183.000000,2219.949951,2171.050049,2175.850098,2152.651123,3039571
2020-07-10,2170.000000,2232.000000,2167.449951,2223.800049,2200.089844,3179516
2020-07-13,2239.000000,2275.000000,2235.000000,2265.250000,2241.097900,4556662
2020-07-14,2264.800049,2269.949951,2226.600098,2234.750000,2210.923096,2403256
2020-07-15,2237.250000,2291.000000,2235.050049,2275.149902,2250.892090,2474512
2020-07-16,2275.250000,2291.300049,2238.000000,2287.850098,2263.457031,2495150
2020-07-17,2285.000000,2340.000000,2281.000000,2334.550049,2309.658936,3088712
2020-07-20,2334.550049,2343.300049,2300.600098,2330.949951,2306.097412,2816219
2020-07-21,2348.000000,2350.000000,2308.000000,2318.000000,2293.285400,3037810
2020-07-22,2330.000000,2330.000000,2241.250000,2248.500000,2224.526367,6950011
2020-07-23,2260.000000,2267.149902,2205.550049,2211.350098,2187.772705,3936886
2020-07-24,2200.000000,2220.000000,2168.050049,2209.300049,2185.744385,4203314
2020-07-27,2217.750000,2235.949951,2184.750000,2221.500000,2197.814453,2209409
2020-07-28,2230.949951,2244.899902,2211.050049,2242.550049,2218.639893,1567837
2020-07-29,2235.000000,2262.199951,2215.000000,2231.300049,2207.510010,2404978
2020-07-30,2231.300049,2238.050049,2190.100098,2195.600098,2181.478516,2044966
2020-07-31,2197.500000,2221.949951,2178.000000,2209.899902,2195.686279,1715659
2020-08-03,2209.800049,2223.050049,2188.550049,2204.500000,2190.321289,1683773
2020-08-04,2209.899902,2218.899902,2175.100098,2197.800049,2183.664307,1981720
2020-08-05,2202.000000,2209.899902,2181.050049,2194.550049,2180.435303,1715477
2020-08-06,2200.000000,2229.000000,2192.050049,2220.449951,2206.168457,1824483
2020-08-07,2223.000000,2234.800049,2205.000000,2210.550049,2196.332275,1777905
2020-08-10,2218.000000,2226.750000,2198.800049,2206.750000,2192.556641,1476289
2020-08-11,2209.050049,2233.000000,2206.000000,2210.649902,2196.431641,1665204
2020-08-12,2200.050049,2209.050049,2190.000000,2195.100098,2180.981689,1170430
2020-08-13,2200.000000,2217.000000,2190.949951,2198.649902,2184.508789,1453227
2020-08-14,2204.000000,2216.000000,2165.050049,2173.899902,2159.917969,1341664
2020-08-17,2181.000000,2203.649902,2168.399902,2196.050049,2181.925537,1559579
2020-08-18,2204.000000,2216.800049,2189.000000,2214.350098,2200.107910,1339001
2020-08-19,2206.000000,2221.500000,2190.000000,2193.600098,2179.491455,1585107
2020-08-20,2186.000000,2197.800049,2176.149902,2185.699951,2171.642090,1768939
2020-08-21,2199.000000,2208.699951,2187.449951,2202.050049,2187.886963,1374411
2020-08-24,2206.000000,2211.649902,2191.199951,2194.399902,2180.286133,1665474
2020-08-25,2194.399902,2197.899902,2175.000000,2179.100098,2165.084717,1774886
2020-08-26,2182.100098,2186.000000,2160.000000,2182.800049,2168.760742,2148146
2020-08-27,2182.100098,2188.000000,2165.000000,2171.250000,2157.285156,2172203
2020-08-28,2179.000000,2182.399902,2145.000000,2151.949951,2138.109131,1939520
2020-08-31,2152.000000,2167.500000,2100.649902,2117.350098,2103.731934,3534275
2020-09-01,2149.949951,2174.899902,2111.750000,2161.649902,2147.746582,1738003
2020-09-02,2171.000000,2185.000000,2136.550049,2141.649902,2127.875244,1667810
2020-09-03,2153.000000,2154.449951,2128.050049,2139.149902,2125.391357,1590711
2020-09-04,2115.000000,2140.899902,2114.000000,2124.649902,2110.984619,1590491
2020-09-07,2126.000000,2174.000000,2103.750000,2162.600098,2148.690674,2362992
2020-09-08,2154.000000,2171.350098,2131.149902,2142.250000,2128.471680,1804209
2020-09-09,2122.000000,2144.199951,2110.000000,2132.250000,2118.535889,2009898
2020-09-10,2142.000000,2143.600098,2118.199951,2133.899902,2120.175293,2137794
2020-09-11,2125.000000,2169.000000,2124.500000,2159.250000,2145.362305,1934110
2020-09-14,2167.000000,2169.699951,2115.000000,2123.500000,2109.842041,2096908
2020-09-15,2136.899902,2137.000000,2109.000000,2124.449951,2110.785889,1224058
2020-09-16,2125.000000,2158.949951,2112.699951,2143.500000,2129.713623,2087841
2020-09-17,2135.000000,2145.050049,2122.250000,2128.199951,2114.511963,1741644
2020-09-18,2120.000000,2127.550049,2090.000000,2098.699951,2085.201660,4475166
2020-09-21,2098.699951,2099.000000,2022.099976,2037.800049,2024.693359,3483963
2020-09-22,2040.550049,2054.750000,2000.050049,2026.800049,2013.764160,2345687
2020-09-23,2035.000000,2060.000000,2025.849976,2052.550049,2039.348511,2410295
2020-09-24,2044.000000,2081.000000,2035.050049,2060.899902,2047.644653,3798089
2020-09-25,2082.000000,2111.000000,2070.850098,2079.050049,2065.678223,2528873
2020-09-28,2083.000000,2092.350098,2060.000000,2063.300049,2050.029297,1523704
2020-09-29,2065.000000,2069.250000,2024.750000,2033.199951,2020.122925,2298844
2020-09-30,2038.949951,2097.000000,2034.000000,2068.250000,2054.947510,2417367
2020-10-01,2090.000000,2108.899902,2070.000000,2095.000000,2081.525391,2131531
2020-10-05,2100.000000,2123.300049,2082.199951,2111.050049,2097.472168,1646110
2020-10-06,2119.000000,2140.000000,2108.050049,2115.899902,2102.291016,1679499
2020-10-07,2123.000000,2150.000000,2113.000000,2139.350098,2125.590332,2428544
2020-10-08,2147.000000,2165.000000,2146.250000,2160.800049,2146.902344,1617365
2020-10-09,2150.000000,2166.949951,2121.100098,2139.649902,2125.888184,2001026
2020-10-12,2145.199951,2147.600098,2123.300049,2138.399902,2124.646240,1741166
2020-10-13,2137.949951,2153.250000,2128.000000,2138.199951,2124.447510,1679160
2020-10-14,2132.600098,2165.000000,2131.000000,2158.149902,2144.269287,2123033
2020-10-15,2160.000000,2179.699951,2141.050049,2152.550049,2138.705322,2344437
2020-10-16,2171.899902,2171.899902,2144.050049,2150.550049,2136.718262,1640399
2020-10-19,2153.550049,2185.000000,2152.100098,2177.800049,2163.792969,1605969
2020-10-20,2182.100098,2208.500000,2132.550049,2173.100098,2159.123291,5855290
2020-10-21,2190.000000,2196.750000,2136.000000,2163.149902,2149.237061,3954242
2020-10-22,2160.000000,2187.250000,2153.199951,2179.350098,2165.333008,2632958
2020-10-23,2188.899902,2188.899902,2140.000000,2143.850098,2130.061279,2805684
2020-10-26,2150.000000,2172.000000,2142.300049,2166.750000,2152.813965,1646050
2020-10-27,2157.000000,2189.000000,2152.449951,2176.699951,2162.699951,2334808
2020-10-28,2183.949951,2183.949951,2144.100098,2164.350098,2164.350098,1555483
2020-10-29,2142.000000,2160.000000,2116.000000,2122.100098,2122.100098,1934367
2020-10-30,2118.199951,2120.000000,2068.000000,2071.300049,2071.300049,2414313
2020-11-02,2072.000000,2087.649902,2056.000000,2072.000000,2072.000000,1486077
2020-11-03,2091.800049,2091.800049,2043.000000,2055.699951,2055.699951,2227402
2020-11-04,2050.000000,2085.000000,2049.949951,2060.050049,2060.050049,1479878
2020-11-05,2087.000000,2112.000000,2061.300049,2102.350098,2102.350098,2083827
2020-11-06,2105.000000,2123.949951,2086.000000,2094.149902,2094.149902,2127821
2020-11-09,2120.000000,2137.649902,2108.050049,2132.550049,2132.550049,2194404
2020-11-10,2140.000000,2144.199951,2092.500000,2136.600098,2136.600098,2471703
2020-11-11,2136.600098,2138.600098,2104.600098,2131.149902,2131.149902,2759942
2020-11-12,2129.000000,2204.000000,2123.250000,2192.100098,2192.100098,3010512
2020-11-13,2205.000000,2218.399902,2178.250000,2185.350098,2185.350098,2799165
2020-11-17,2220.350098,2220.350098,2172.000000,2177.600098,2177.600098,2733542
2020-11-18,2170.000000,2170.000000,2125.899902,2132.100098,2132.100098,3315051
2020-11-19,2125.800049,2147.399902,2114.500000,2130.750000,2130.750000,2704650
2020-11-20,2135.000000,2142.000000,2100.500000,2120.250000,2120.250000,2827038
2020-11-23,2140.000000,2140.000000,2095.000000,2129.399902,2129.399902,3244576
2020-11-24,2132.000000,2161.000000,2122.399902,2157.350098,2157.350098,1989749
2020-11-25,2162.000000,2171.000000,2128.000000,2135.850098,2135.850098,1786536
2020-11-26,2140.000000,2162.000000,2122.550049,2154.199951,2154.199951,1630887
2020-11-27,2167.000000,2167.500000,2126.000000,2138.199951,2138.199951,5687861
2020-12-01,2159.000000,2159.000000,2129.600098,2133.050049,2133.050049,2226742
2020-12-02,2145.000000,2149.899902,2122.100098,2139.300049,2139.300049,1634682
2020-12-03,2153.000000,2156.399902,2120.100098,2127.750000,2127.750000,3469163
2020-12-04,2141.000000,2195.399902,2133.600098,2184.199951,2184.199951,3712143
2020-12-07,2189.250000,2259.600098,2187.000000,2256.149902,2256.149902,4147649
2020-12-08,2259.500000,2278.199951,2240.149902,2260.000000,2260.000000,2768211
2020-12-09,2260.000000,2296.000000,2243.250000,2292.199951,2292.199951,2123890
2020-12-10,2290.000000,2357.899902,2275.050049,2350.449951,2350.449951,3041370
2020-12-11,2366.250000,2385.000000,2333.000000,2374.750000,2374.750000,2611218
2020-12-14,2380.949951,2391.399902,2356.149902,2370.750000,2370.750000,1575632
2020-12-15,2361.000000,2363.949951,2307.300049,2320.050049,2320.050049,2207176
2020-12-16,2317.100098,2354.899902,2303.000000,2343.949951,2343.949951,1748634
2020-12-17,2342.000000,2342.000000,2310.000000,2315.300049,2315.300049,1912868
2020-12-18,2322.000000,2343.899902,2312.000000,2333.449951,2333.449951,2229891
2020-12-21,2343.000000,2350.000000,2265.100098,2305.949951,2305.949951,2003069
2020-12-22,2310.000000,2333.300049,2281.000000,2311.000000,2311.000000,1764488
2020-12-23,2320.000000,2377.300049,2303.250000,2370.350098,2370.350098,1559556
2020-12-24,2382.500000,2410.000000,2360.050049,2402.250000,2402.250000,2489444
2020-12-28,2405.000000,2417.399902,2376.050049,2388.899902,2388.899902,1532477
2020-12-29,2395.000000,2403.000000,2354.199951,2384.300049,2384.300049,1749013
2020-12-30,2390.000000,2414.649902,2376.000000,2406.600098,2406.600098,1924886
2020-12-31,2396.000000,2417.000000,2387.000000,2395.399902,2395.399902,1709836
2021-01-01,2395.399902,2404.000000,2382.000000,2387.550049,2387.550049,830096
2021-01-04,2405.000000,2430.000000,2400.699951,2426.500000,2426.500000,1614486
2021-01-05,2405.000000,2456.949951,2405.000000,2450.550049,2450.550049,2356303
2021-01-06,2445.800049,2450.000000,2401.600098,2417.300049,2417.300049,1641627
2021-01-07,2432.000000,2432.699951,2365.000000,2368.850098,2368.850098,2403881
2021-01-08,2390.000000,2400.000000,2360.800049,2391.199951,2391.199951,2797948
2021-01-11,2425.000000,2441.949951,2402.100098,2429.100098,2429.100098,2787672
2021-01-12,2429.800049,2433.100098,2372.000000,2375.899902,2375.899902,2036708
2021-01-13,2381.899902,2398.000000,2364.600098,2372.350098,2372.350098,1702740
2021-01-14,2366.000000,2406.000000,2351.149902,2398.550049,2398.550049,2120413
2021-01-15,2392.500000,2407.000000,2342.800049,2351.149902,2351.149902,2042957
2021-01-18,2374.500000,2374.500000,2320.649902,2332.600098,2332.600098,1733797
2021-01-19,2342.000000,2382.100098,2323.250000,2363.149902,2363.149902,1495269
2021-01-20,2357.000000,2373.949951,2346.300049,2355.949951,2355.949951,1380562
2021-01-21,2376.000000,2399.899902,2350.000000,2367.649902,2367.649902,1521133
2021-01-22,2359.000000,2422.000000,2357.000000,2409.350098,2409.350098,2548089
2021-01-25,2420.000000,2432.350098,2385.000000,2399.600098,2399.600098,2124424
2021-01-27,2425.000000,2425.000000,2335.000000,2391.300049,2391.300049,2460297
2021-01-28,2367.899902,2411.550049,2290.000000,2300.399902,2300.399902,3867335
2021-01-29,2315.100098,2322.300049,2251.000000,2263.899902,2263.899902,4472694
2021-02-01,2276.649902,2285.000000,2226.199951,2248.600098,2248.600098,2717591
2021-02-02,2271.050049,2275.000000,2205.000000,2230.449951,2230.449951,4215607
2021-02-03,2231.500000,2257.000000,2217.600098,2232.850098,2232.850098,2353429
2021-02-04,2239.000000,2259.000000,2226.199951,2245.600098,2245.600098,2908003
2021-02-05,2245.600098,2285.000000,2241.000000,2269.550049,2269.550049,4036942
2021-02-08,2288.000000,2288.699951,2230.300049,2236.649902,2236.649902,2780672
2021-02-09,2247.000000,2254.699951,2210.500000,2216.250000,2216.250000,3131481
2021-02-10,2220.000000,2241.000000,2212.750000,2235.500000,2235.500000,2055520
2021-02-11,2242.000000,2268.000000,2234.050049,2263.000000,2263.000000,1824480
2021-02-12,2268.000000,2272.250000,2232.000000,2241.550049,2241.550049,1222094
2021-02-15,2251.000000,2261.750000,2213.000000,2215.199951,2215.199951,2449407
2021-02-16,2227.000000,2229.050049,2190.050049,2196.100098,2196.100098,2396356
2021-02-17,2196.000000,2201.550049,2160.000000,2163.899902,2163.899902,2447956
2021-02-18,2164.000000,2168.550049,2143.199951,2146.850098,2146.850098,3205806
2021-02-19,2152.899902,2194.000000,2150.000000,2181.100098,2181.100098,3364030
2021-02-22,2205.000000,2205.000000,2161.600098,2167.600098,2167.600098,2678023
2021-02-23,2185.000000,2192.350098,2170.199951,2179.250000,2179.250000,2062144
2021-02-24,2188.500000,2188.500000,2155.550049,2178.850098,2178.850098,924143
2021-02-25,2177.000000,2185.000000,2160.000000,2163.300049,2163.300049,2099460
2021-02-26,2158.000000,2182.399902,2120.000000,2132.050049,2132.050049,3413194
2021-03-01,2135.199951,2154.000000,2128.000000,2143.399902,2143.399902,1229276
2021-03-02,2162.000000,2183.899902,2140.550049,2177.949951,2177.949951,2095135
2021-03-03,2205.000000,2205.000000,2173.500000,2193.500000,2193.500000,2031893
2021-03-04,2185.000000,2203.500000,2174.000000,2196.899902,2196.899902,2290915
2021-03-05,2184.600098,2209.949951,2175.250000,2201.100098,2201.100098,2129837
2021-03-08,2210.000000,2216.100098,2169.000000,2174.550049,2174.550049,1440724
2021-03-09,2176.000000,2205.000000,2162.050049,2198.500000,2198.500000,1711842
2021-03-10,2210.500000,2236.899902,2193.199951,2227.850098,2227.850098,2696085
2021-03-12,2236.000000,2239.899902,2200.000000,2204.750000,2204.750000,2038930
2021-03-15,2206.100098,2217.449951,2183.300049,2210.850098,2210.850098,1632022
2021-03-16,2221.350098,2247.500000,2214.300049,2244.050049,2244.050049,2633605
2021-03-17,2254.000000,2262.000000,2219.250000,2226.300049,2226.300049,3703365
2021-03-18,2238.000000,2249.899902,2205.000000,2215.199951,2215.199951,2597091
2021-03-19,2204.100098,2325.250000,2191.500000,2312.050049,2312.050049,5382214
2021-03-22,2310.100098,2356.500000,2302.000000,2353.199951,2353.199951,2471516
2021-03-23,2363.000000,2372.949951,2333.399902,2337.300049,2337.300049,1970612
2021-03-24,2337.300049,2350.899902,2315.550049,2318.600098,2318.600098,1762743
2021-03-25,2318.600098,2322.000000,2231.050049,2237.050049,2237.050049,2495833
2021-03-26,2255.000000,2325.000000,2248.800049,2317.899902,2317.899902,1929236
2021-03-30,2325.000000,2406.000000,2325.000000,2398.750000,2398.750000,3327203
2021-03-31,2386.000000,2438.600098,2381.649902,2431.500000,2431.500000,2709756
2021-04-01,2446.800049,2446.800049,2392.050049,2399.100098,2399.100098,1532653
2021-04-05,2400.000000,2417.199951,2346.500000,2373.199951,2373.199951,2545447
2021-04-06,2382.300049,2411.000000,2377.350098,2408.300049,2408.300049,3683538
2021-04-07,2417.000000,2435.000000,2387.350098,2405.250000,2405.250000,1713970
2021-04-08,2406.000000,2439.000000,2395.600098,2409.899902,2409.899902,1814515
2021-04-09,2424.000000,2483.000000,2424.000000,2476.000000,2476.000000,2604858
2021-04-12,2441.500000,2488.250000,2437.600098,2462.000000,2462.000000,2261945
2021-04-13,2470.000000,2505.899902,2448.800049,2454.399902,2454.399902,1811781
2021-04-15,2497.000000,2497.000000,2421.100098,2460.649902,2460.649902,3352379
2021-04-16,2469.899902,2478.949951,2440.000000,2458.899902,2458.899902,1225517
2021-04-19,2440.000000,2459.699951,2414.050049,2436.649902,2436.649902,1746316
2021-04-20,2456.000000,2456.000000,2383.800049,2397.449951,2397.449951,1589508
2021-04-22,2390.000000,2390.000000,2332.000000,2349.399902,2349.399902,1721265
2021-04-23,2340.000000,2350.000000,2300.000000,2307.750000,2307.750000,1949756
2021-04-26,2324.000000,2369.800049,2311.000000,2360.649902,2360.649902,1946867
2021-04-27,2363.000000,2386.449951,2352.949951,2379.850098,2379.850098,1668456
2021-04-28,2380.000000,2445.000000,2366.300049,2406.550049,2406.550049,1951215
2021-04-29,2433.000000,2448.949951,2384.149902,2407.600098,2407.600098,3151150
2021-04-30,2410.000000,2442.899902,2330.250000,2353.750000,2353.750000,4010778
2021-05-03,2343.000000,2415.000000,2340.000000,2408.000000,2408.000000,1692239
2021-05-04,2409.199951,2418.199951,2372.199951,2407.649902,2407.649902,1453111
2021-05-05,2434.000000,2434.000000,2368.649902,2394.649902,2394.649902,1256857
2021-05-06,2400.000000,2416.550049,2377.050049,2390.000000,2390.000000,1377740
2021-05-07,2408.800049,2424.000000,2397.149902,2411.199951,2411.199951,1169518
2021-05-10,2423.000000,2446.949951,2416.000000,2432.899902,2432.899902,988495
2021-05-11,2425.000000,2426.000000,2394.250000,2402.100098,2402.100098,1019077
2021-05-12,2389.000000,2389.000000,2325.050049,2328.949951,2328.949951,2053932
2021-05-14,2328.899902,2382.399902,2326.000000,2376.850098,2376.850098,1388970
2021-05-17,2385.000000,2395.000000,2366.500000,2372.949951,2372.949951,524484'''

nestle_csv = '''Date,Open,High,Low,Close,Adj Close,Volume
2020-05-18,16441.000000,16575.000000,16162.200195,16240.349609,15965.980469,167960
2020-05-19,16280.200195,16415.800781,16108.650391,16150.299805,15877.452148,118144
2020-05-20,16241.000000,16594.949219,16151.000000,16303.400391,16027.966797,179539
2020-05-21,16330.000000,16505.000000,16265.000000,16295.450195,16020.150391,121080
2020-05-22,16296.000000,16396.800781,16150.000000,16215.150391,15941.207031,118300
2020-05-26,16380.000000,16750.000000,16324.400391,16655.150391,16373.772461,198692
2020-05-27,16700.000000,16820.000000,16487.099609,16716.500000,16434.085938,167533
2020-05-28,16716.500000,17030.000000,16680.000000,16989.150391,16702.132813,176288
2020-05-29,16950.000000,17600.000000,16784.550781,17540.800781,17244.460938,244466
2020-06-01,17540.800781,17749.000000,17111.000000,17183.449219,16893.146484,175186
2020-06-02,17477.000000,17477.000000,16972.000000,17014.449219,16727.001953,132201
2020-06-03,17175.000000,17521.949219,17152.000000,17407.550781,17113.460938,146101
2020-06-04,17406.900391,17546.550781,17148.050781,17196.599609,16906.076172,97631
2020-06-05,17317.000000,17460.000000,16956.550781,17096.849609,16808.011719,162096
2020-06-08,17250.000000,17250.000000,16790.000000,16836.199219,16551.763672,147070
2020-06-09,16652.000000,16985.000000,16521.000000,16552.550781,16272.906250,174685
2020-06-10,16650.000000,16832.000000,16410.000000,16534.699219,16255.358398,123440
2020-06-11,16550.000000,16850.000000,16535.750000,16608.250000,16388.126953,165131
2020-06-12,16300.000000,16717.099609,16300.000000,16496.300781,16277.660156,140678
2020-06-15,16500.000000,16650.000000,16370.000000,16400.300781,16182.931641,139654
2020-06-16,16598.000000,16598.000000,16270.000000,16342.099609,16125.501953,193237
2020-06-17,16345.000000,16440.000000,16115.150391,16198.150391,15983.460938,126959
2020-06-18,16218.500000,16300.000000,16054.599609,16178.950195,15964.515625,158445
2020-06-19,16290.000000,16305.650391,16052.200195,16094.849609,15881.529297,168893
2020-06-22,16290.000000,16290.000000,16065.000000,16140.799805,15926.871094,135915
2020-06-23,16289.799805,16720.000000,16210.049805,16548.449219,16329.117188,244735
2020-06-24,16660.000000,16860.000000,16519.150391,16707.949219,16486.503906,201069
2020-06-25,16590.300781,16850.000000,16565.000000,16787.000000,16564.505859,169665
2020-06-26,16850.000000,16930.000000,16630.000000,16677.500000,16456.457031,110341
2020-06-29,16740.000000,16980.599609,16540.000000,16724.849609,16503.179688,169646
2020-06-30,16799.000000,17298.000000,16753.900391,17174.449219,16946.818359,201033
2020-07-01,17205.000000,17205.000000,16720.000000,16798.849609,16576.199219,216900
2020-07-02,16985.000000,16985.000000,16661.099609,16789.349609,16566.824219,146970
2020-07-03,16813.800781,16990.000000,16650.000000,16695.599609,16474.318359,181658
2020-07-06,16755.000000,16842.000000,16686.949219,16802.000000,16579.306641,98337
2020-07-07,16855.000000,16950.000000,16730.050781,16912.400391,16688.246094,141356
2020-07-08,16936.000000,17150.000000,16737.000000,16802.050781,16579.357422,218721
2020-07-09,16840.000000,16965.949219,16681.000000,16759.099609,16536.974609,91894
2020-07-10,16759.099609,16874.949219,16628.000000,16833.099609,16609.996094,133611
2020-07-13,16885.000000,17098.000000,16875.050781,16956.849609,16732.103516,114029
2020-07-14,16949.000000,17115.150391,16852.000000,16895.800781,16671.865234,75853
2020-07-15,16940.000000,17143.000000,16740.000000,16908.750000,16684.642578,126519
2020-07-16,16926.000000,17764.599609,16760.000000,17495.650391,17263.765625,296537
2020-07-17,17494.000000,17620.000000,17165.000000,17229.949219,17001.583984,125138
2020-07-20,17324.000000,17370.000000,17070.599609,17247.000000,17018.410156,196574
2020-07-21,17350.000000,17494.949219,17208.050781,17275.599609,17046.628906,211186
2020-07-22,17350.000000,17440.000000,17001.050781,17151.849609,16924.521484,156478
2020-07-23,17120.000000,17298.949219,17050.000000,17256.400391,17027.685547,106941
2020-07-24,17125.099609,17410.000000,16960.000000,17251.849609,17023.195313,207280
2020-07-27,17300.000000,17414.949219,17211.900391,17350.250000,17120.291016,103828
2020-07-28,17407.000000,17474.650391,17007.699219,17095.400391,16868.820313,158967
2020-07-29,16999.949219,17199.000000,16510.599609,16597.849609,16377.863281,450505
2020-07-30,16740.000000,16771.949219,16451.099609,16524.000000,16304.992188,128484
2020-07-31,16526.000000,16674.250000,16432.199219,16522.199219,16303.214844,147940
2020-08-03,16528.000000,16600.000000,16350.000000,16524.800781,16305.783203,115843
2020-08-04,16635.300781,16648.550781,16413.849609,16580.000000,16360.250000,110048
2020-08-05,16630.000000,16643.949219,16460.000000,16503.349609,16284.615234,97765
2020-08-06,16600.000000,16700.000000,16450.000000,16677.150391,16456.111328,105888
2020-08-07,16668.900391,16899.000000,16555.900391,16788.599609,16566.083984,88450
2020-08-10,16789.949219,16859.949219,16640.000000,16665.750000,16444.863281,64144
2020-08-11,16725.000000,16785.650391,16581.000000,16622.800781,16402.484375,37873
2020-08-12,16582.050781,16622.050781,16450.000000,16540.849609,16321.619141,82163
2020-08-13,16462.000000,16661.949219,16458.400391,16474.199219,16255.851563,66303
2020-08-14,16487.000000,16549.000000,16305.049805,16354.700195,16137.936523,59434
2020-08-17,16450.000000,16524.500000,16313.750000,16422.900391,16205.232422,62584
2020-08-18,16400.000000,16624.599609,16311.000000,16554.599609,16335.185547,104400
2020-08-19,16500.000000,16553.250000,16315.500000,16345.750000,16129.104492,125005
2020-08-20,16300.000000,16340.000000,16185.250000,16211.950195,15997.078125,98085
2020-08-21,16322.500000,16690.000000,16250.000000,16561.949219,16342.439453,144143
2020-08-24,16580.000000,16707.900391,16401.000000,16478.800781,16260.391602,74072
2020-08-25,16456.000000,16477.949219,16265.000000,16288.200195,16072.317383,108867
2020-08-26,16290.500000,16348.349609,16170.000000,16190.049805,15975.467773,116508
2020-08-27,16235.000000,16240.299805,16085.000000,16113.700195,15900.130859,127645
2020-08-28,16156.000000,16368.000000,16090.000000,16101.250000,15887.844727,148489
2020-08-31,16215.000000,16287.000000,15875.000000,15949.549805,15738.155273,221293
2020-09-01,15995.000000,16254.950195,15739.900391,16196.900391,15982.227539,176947
2020-09-02,16191.000000,16260.000000,15970.000000,15992.549805,15780.585938,106192
2020-09-03,16050.000000,16439.599609,15921.250000,16393.750000,16176.467773,133887
2020-09-04,16155.000000,16423.900391,16150.000000,16232.599609,16017.454102,98932
2020-09-07,16241.000000,16515.000000,15972.099609,16403.500000,16186.089844,104808
2020-09-08,16369.599609,16369.599609,16100.000000,16126.500000,15912.759766,105414
2020-09-09,16000.000000,16350.000000,16000.000000,16312.500000,16096.294922,135800
2020-09-10,16261.250000,16450.000000,16201.250000,16420.699219,16203.060547,77208
2020-09-11,16400.000000,16400.000000,16204.000000,16325.349609,16108.974609,81228
2020-09-14,16400.000000,16400.000000,16040.049805,16083.299805,15870.132813,102926
2020-09-15,16090.000000,16165.900391,16012.000000,16040.950195,15828.344727,94984
2020-09-16,16066.500000,16225.000000,16050.000000,16197.799805,15983.115234,124262
2020-09-17,16240.000000,16298.799805,16090.000000,16121.349609,15907.678711,100681
2020-09-18,16250.000000,16250.000000,15925.000000,16087.049805,15873.833008,197351
2020-09-21,16025.000000,16049.500000,15154.500000,15430.750000,15226.232422,277165
2020-09-22,15430.000000,15430.000000,15100.000000,15199.250000,14997.799805,107120
2020-09-23,15200.049805,15398.950195,15130.000000,15366.049805,15162.388672,131430
2020-09-24,15348.000000,15438.299805,15175.000000,15258.849609,15056.610352,111275
2020-09-25,15495.000000,15830.000000,15301.000000,15736.799805,15528.224609,178269
2020-09-28,15820.000000,15870.549805,15620.150391,15719.799805,15511.451172,54411
2020-09-29,15800.000000,15851.049805,15575.000000,15620.849609,15413.812500,48179
2020-09-30,15629.950195,16028.750000,15605.049805,15918.650391,15707.666016,103500
2020-10-01,15919.000000,16050.000000,15797.599609,16000.500000,15788.430664,62638
2020-10-05,15998.000000,16140.000000,15732.349609,16033.150391,15820.647461,71081
2020-10-06,16033.000000,16128.000000,15834.049805,15876.849609,15666.418945,75278
2020-10-07,15956.200195,16124.950195,15854.650391,16114.400391,15900.821289,99647
2020-10-08,16100.000000,16330.000000,16052.049805,16304.599609,16088.499023,78576
2020-10-09,16250.000000,16350.000000,15985.000000,16034.099609,15821.584961,74246
2020-10-12,16100.000000,16149.599609,15860.000000,15902.200195,15691.433594,58166
2020-10-13,15880.000000,15979.250000,15652.000000,15673.000000,15465.270508,76235
2020-10-14,15674.000000,15820.000000,15550.000000,15673.049805,15465.320313,66595
2020-10-15,15700.000000,15788.000000,15561.799805,15602.750000,15395.952148,50244
2020-10-16,15650.000000,15820.400391,15375.000000,15430.250000,15225.738281,132361
2020-10-19,15550.000000,16178.000000,15425.049805,16093.150391,15879.853516,182794
2020-10-20,15900.000000,16410.000000,15850.000000,16252.549805,16037.138672,203344
2020-10-21,16240.000000,16314.450195,15942.049805,16030.849609,15818.377930,150091
2020-10-22,15990.049805,16081.099609,15768.049805,15909.450195,15698.586914,113580
2020-10-23,15930.000000,16066.000000,15807.250000,15865.450195,15655.169922,73595
2020-10-26,16200.000000,16375.000000,16052.150391,16259.349609,16043.849609,346006
2020-10-27,16480.000000,17300.000000,16395.550781,17229.849609,17001.486328,493567
2020-10-28,17150.000000,17295.000000,16934.300781,17119.150391,16892.253906,163262
2020-10-29,16780.050781,17215.949219,16711.849609,16870.599609,16779.318359,136561
2020-10-30,16891.000000,17360.000000,16890.000000,17161.599609,17068.744141,239316
2020-11-02,17201.000000,17360.000000,16940.000000,17114.949219,17022.345703,84764
2020-11-03,17239.000000,17269.949219,16760.000000,16892.500000,16801.099609,104743
2020-11-04,17000.000000,17200.000000,16825.000000,16997.099609,16905.134766,97909
2020-11-05,17100.000000,17399.000000,17020.800781,17221.300781,17128.121094,101747
2020-11-06,17240.000000,17279.849609,16903.099609,17006.650391,16914.632813,119003
2020-11-09,17148.000000,17370.000000,17002.000000,17332.000000,17238.222656,111432
2020-11-10,17332.000000,17332.000000,16582.849609,16604.750000,16514.906250,289666
2020-11-11,16670.000000,16828.050781,16475.000000,16697.599609,16607.253906,248311
2020-11-12,16799.000000,16900.000000,16533.599609,16718.599609,16628.140625,147831
2020-11-13,16725.000000,16890.000000,16650.000000,16825.650391,16734.613281,74273
2020-11-17,16950.000000,17065.849609,16853.050781,16982.849609,16890.960938,119504
2020-11-18,16950.000000,16969.400391,16671.849609,16872.599609,16781.306641,144633
2020-11-19,16840.000000,17200.000000,16786.050781,16934.150391,16842.525391,180222
2020-11-20,17050.000000,17500.000000,16915.150391,17460.400391,17365.927734,254010
2020-11-23,17461.000000,17848.900391,17460.400391,17786.400391,17690.164063,202645
2020-11-24,17750.000000,17971.199219,17600.000000,17677.599609,17581.951172,214497
2020-11-25,17789.000000,17789.000000,17340.000000,17520.199219,17425.402344,106508
2020-11-26,17599.000000,17839.000000,17280.000000,17741.949219,17645.953125,145376
2020-11-27,17850.000000,18068.900391,16949.750000,17888.949219,17792.158203,1106746
2020-12-01,18067.800781,18067.900391,17331.550781,17418.150391,17323.906250,235086
2020-12-02,17418.150391,17449.800781,17150.000000,17266.449219,17173.025391,161952
2020-12-03,17450.000000,17540.050781,17316.000000,17492.150391,17397.505859,127275
2020-12-04,17550.000000,17877.500000,17520.000000,17828.699219,17732.234375,124700
2020-12-07,17651.000000,17900.000000,17449.849609,17562.550781,17467.525391,126030
2020-12-08,17674.000000,17729.400391,17386.250000,17519.449219,17424.658203,118699
2020-12-09,17543.599609,17641.699219,17481.150391,17569.150391,17474.089844,89503
2020-12-10,17607.000000,18404.000000,17562.199219,18352.699219,18253.398438,304461
2020-12-11,18300.000000,18460.000000,18200.000000,18401.099609,18301.537109,162941
2020-12-14,18354.699219,18679.000000,18300.000000,18591.800781,18491.207031,130713
2020-12-15,18460.300781,18550.000000,18130.849609,18189.900391,18091.480469,135755
2020-12-16,18398.000000,18734.949219,18250.150391,18323.449219,18224.306641,221607
2020-12-17,18338.000000,18395.150391,18136.800781,18305.349609,18206.304688,98104
2020-12-18,18395.000000,18599.000000,18158.000000,18362.750000,18263.394531,249790
2020-12-21,18262.750000,18525.000000,18101.000000,18192.599609,18094.166016,142222
2020-12-22,18200.000000,18665.000000,18000.000000,18553.550781,18453.164063,132774
2020-12-23,18555.050781,18812.000000,18514.949219,18732.699219,18631.341797,102845
2020-12-24,18612.199219,18830.000000,18372.599609,18566.349609,18465.892578,125437
2020-12-28,18844.000000,18844.000000,18520.000000,18597.349609,18496.724609,65357
2020-12-29,18620.000000,18700.000000,18175.400391,18262.650391,18163.837891,198714
2020-12-30,18399.500000,18468.000000,18230.000000,18379.250000,18279.806641,93654
2020-12-31,18278.050781,18450.000000,18257.500000,18390.250000,18290.746094,85442
2021-01-01,18389.699219,18520.000000,18306.050781,18450.699219,18350.869141,50286
2021-01-04,18495.000000,18630.000000,18329.750000,18377.949219,18278.511719,62035
2021-01-05,18343.500000,18620.000000,18278.050781,18558.250000,18457.837891,83673
2021-01-06,18510.050781,18600.000000,18430.000000,18515.250000,18415.070313,57706
2021-01-07,18599.000000,18615.000000,18050.099609,18127.300781,18029.220703,120605
2021-01-08,18227.300781,18350.000000,18000.000000,18306.250000,18207.201172,151635
2021-01-11,18350.000000,18545.849609,18188.099609,18391.449219,18291.939453,114555
2021-01-12,18410.000000,18439.949219,17950.000000,17999.099609,17901.712891,141566
2021-01-13,18039.949219,18275.000000,17965.000000,18019.300781,17921.804688,109387
2021-01-14,18049.000000,18229.400391,18049.000000,18132.599609,18034.490234,58488
2021-01-15,18110.050781,18249.000000,17800.000000,17834.000000,17737.505859,96401
2021-01-18,17849.949219,18074.000000,17556.750000,17630.699219,17535.304688,120674
2021-01-19,17730.000000,17820.500000,17670.000000,17754.699219,17658.634766,62849
2021-01-20,17767.000000,17940.000000,17623.050781,17681.300781,17585.632813,73831
2021-01-21,17770.000000,17900.000000,17655.349609,17716.300781,17620.443359,70977
2021-01-22,17784.000000,17882.300781,17550.000000,17589.599609,17494.427734,98250
2021-01-25,17652.000000,17700.000000,17229.849609,17415.250000,17321.021484,116436
2021-01-27,17472.949219,17597.699219,17340.250000,17538.099609,17443.207031,66402
2021-01-28,17470.000000,17520.000000,17203.000000,17387.650391,17293.572266,89055
2021-01-29,17385.050781,17467.849609,16961.300781,17060.000000,16967.693359,112888
2021-02-01,17200.000000,17275.699219,17000.000000,17087.699219,16995.244141,84406
2021-02-02,17150.050781,17350.000000,17054.500000,17190.300781,17097.289063,130298
2021-02-03,17200.050781,17289.000000,17051.000000,17166.400391,17073.519531,112325
2021-02-04,17199.900391,17226.949219,17052.449219,17074.000000,16981.619141,92100
2021-02-05,17131.050781,17167.199219,17003.000000,17115.750000,17023.142578,98097
2021-02-08,17180.000000,17284.900391,17101.150391,17218.199219,17125.037109,121607
2021-02-09,17211.000000,17510.000000,17150.000000,17318.949219,17225.242188,121012
2021-02-10,17340.199219,17436.800781,17086.500000,17177.349609,17084.408203,116576
2021-02-11,17189.900391,17447.449219,17150.000000,17412.250000,17318.037109,65366
2021-02-12,17410.099609,17519.000000,17240.000000,17294.300781,17200.726563,54528
2021-02-15,17349.000000,17506.750000,17258.050781,17479.849609,17385.271484,110119
2021-02-16,17555.000000,17646.000000,17139.099609,17217.250000,17124.093750,229774
2021-02-17,16900.000000,16900.000000,16350.000000,16741.199219,16650.617188,599343
2021-02-18,16756.000000,16900.000000,16300.000000,16364.700195,16276.156250,251066
2021-02-19,16430.000000,16464.699219,16212.950195,16376.400391,16287.792969,130275
2021-02-22,16411.050781,16530.000000,16024.599609,16096.299805,16009.208008,214915
2021-02-23,16131.000000,16255.000000,16002.099609,16169.950195,16082.459961,142998
2021-02-24,16283.000000,16378.000000,16100.000000,16346.150391,16257.707031,84342
2021-02-25,16306.000000,16373.299805,16100.000000,16113.900391,16026.712891,185713
2021-02-26,16050.500000,16284.400391,16003.900391,16101.599609,16014.479492,226873
2021-03-01,16236.000000,16367.599609,16100.000000,16292.450195,16204.296875,90718
2021-03-02,16302.000000,16620.000000,16289.750000,16576.150391,16486.462891,123097
2021-03-03,16705.000000,16720.000000,16546.650391,16699.250000,16608.896484,92873
2021-03-04,16656.550781,16707.949219,16445.000000,16661.800781,16571.648438,129981
2021-03-05,16516.000000,16840.000000,16456.000000,16782.900391,16692.093750,102741
2021-03-08,16725.050781,16900.000000,16607.000000,16852.449219,16761.265625,91842
2021-03-09,16910.199219,16949.949219,16660.000000,16910.800781,16819.302734,92717
2021-03-10,16949.699219,17000.000000,16785.000000,16931.449219,16839.837891,63788
2021-03-12,16977.000000,17009.849609,16650.000000,16722.449219,16631.968750,84914
2021-03-15,16879.849609,16879.849609,16630.000000,16833.050781,16741.972656,68983
2021-03-16,16835.000000,17020.000000,16695.000000,16741.949219,16651.363281,84470
2021-03-17,16790.000000,16790.000000,16340.049805,16455.750000,16366.712891,157173
2021-03-18,16467.900391,16624.949219,16180.000000,16201.250000,16113.590820,96365
2021-03-19,16200.000000,16595.000000,16070.000000,16450.849609,16361.839844,199416
2021-03-22,16451.000000,16675.000000,16351.000000,16588.050781,16498.298828,108786
2021-03-23,16575.000000,16750.000000,16506.400391,16681.349609,16591.091797,100358
2021-03-24,16600.000000,16769.800781,16419.250000,16448.099609,16359.104492,75555
2021-03-25,16425.000000,16584.500000,16116.000000,16170.099609,16082.608398,108704
2021-03-26,16299.000000,16618.449219,16254.000000,16571.750000,16482.085938,126578
2021-03-30,16610.000000,17253.550781,16610.000000,17165.000000,17072.125000,148902
2021-03-31,17299.000000,17299.000000,16951.550781,17165.199219,17072.324219,110331
2021-04-01,17165.199219,17188.550781,16967.599609,17083.150391,16990.718750,83176
2021-04-05,17150.000000,17150.000000,16723.900391,16880.550781,16789.214844,81557
2021-04-06,16910.000000,17120.000000,16800.000000,17081.699219,16989.275391,64193
2021-04-07,17100.000000,17865.949219,17025.000000,17409.849609,17315.650391,178465
2021-04-08,17425.050781,17759.000000,17326.849609,17689.900391,17594.185547,118006
2021-04-09,17611.099609,17849.000000,17500.000000,17592.400391,17497.212891,73944
2021-04-12,17375.150391,17620.000000,17341.099609,17501.699219,17407.003906,57532
2021-04-13,17587.000000,17599.949219,17040.099609,17168.849609,17075.955078,110527
2021-04-15,17325.000000,17400.000000,16775.000000,16875.650391,16784.341797,118314
2021-04-16,16969.000000,17335.000000,16862.800781,17252.099609,17158.753906,94190
2021-04-19,17201.050781,17240.000000,16950.000000,17089.949219,16997.480469,50938
2021-04-20,17420.000000,17420.000000,16890.050781,17095.650391,17003.152344,103687
2021-04-22,17190.000000,17190.000000,16551.250000,16790.449219,16699.601563,204431
2021-04-23,16785.000000,16799.300781,16605.000000,16625.699219,16535.742188,68600
2021-04-26,16799.050781,16895.000000,16663.650391,16870.599609,16779.318359,152362
2021-04-27,16899.000000,16930.000000,16750.000000,16779.000000,16688.214844,44651
2021-04-28,16775.000000,16831.099609,16611.000000,16633.800781,16543.800781,86876
2021-04-29,16679.000000,16714.599609,16480.000000,16502.550781,16502.550781,80207
2021-04-30,16500.000000,16569.199219,16255.000000,16309.250000,16309.250000,97783
2021-05-03,16360.000000,16574.949219,16272.650391,16506.750000,16506.750000,64048
2021-05-04,16510.250000,16620.000000,16380.000000,16546.599609,16546.599609,105796
2021-05-05,16561.050781,16610.000000,16457.449219,16549.250000,16549.250000,35322
2021-05-06,16525.050781,16769.800781,16525.050781,16717.599609,16717.599609,70319
2021-05-07,16788.550781,16798.099609,16665.000000,16761.349609,16761.349609,37666
2021-05-10,16830.000000,16850.000000,16670.000000,16787.300781,16787.300781,47658
2021-05-11,16749.000000,16949.000000,16710.349609,16870.099609,16870.099609,60063
2021-05-12,16770.250000,16857.400391,16620.000000,16717.900391,16717.900391,60576
2021-05-14,16717.000000,17260.000000,16605.000000,17224.150391,17224.150391,96582
2021-05-17,17220.000000,17300.000000,17005.000000,17167.000000,17167.000000,38613'''

r1 = get_return(hind_csv)
r2 = get_return(nestle_csv)

print("Expected Returns of HINDUNILVR is {:.2f}%".format(r1*100))
print("Expected Returns of NESTLEIND  is {:.2f}%".format(r2*100))

v1 = get_volatility(hind_csv)
v2 = get_volatility(nestle_csv)

print("Annualized Volatility of HINDUNILVR is {:.2f}%".format(v1*100))
print("Annualized Volatility of NESTLEIND  is {:.2f}%".format(v2*100))

Expected Returns of HINDUNILVR is 18.59%
Expected Returns of NESTLEIND  is 7.25%
Annualized Volatility of HINDUNILVR is 20.61%
Annualized Volatility of NESTLEIND  is 21.81%
